In [19]:
import pickle # used to save and restore python objects
import gzip
import numpy as np

# Load the dataset
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding = 'latin1')
f.close()

# gather intelligence on the data
print(type(train_set))
print(type(test_set))

<class 'tuple'>
<class 'tuple'>


In [20]:
# take a closer look
print("Inputs:")
print(train_set[0])
print("Inputs shape is " + str(train_set[0].shape))
print("Input type is " + str(type(train_set[0])))
print("Labels:")
print(train_set[1])
print("Labels shape is" + str(train_set[1].shape))
print("Labels type is " + str(type(train_set[1])))


Inputs:
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Inputs shape is (50000, 784)
Input type is <class 'numpy.ndarray'>
Labels:
[5 0 4 ..., 8 4 8]
Labels shape is(50000,)
Labels type is <class 'numpy.ndarray'>


In [21]:
import numpy as np
def to_categorical(y, nb_classes):
    """ to_categorical.

    Convert class vector (integers from 0 to nb_classes)
    to binary class matrix, for use with categorical_crossentropy.

    Arguments:
        y: `array`. Class vector to convert.
        nb_classes: `int`. Total number of classes.

    """
    y = np.asarray(y, dtype='int32')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    Y[np.arange(len(y)),y] = 1.
    return Y

In [22]:
train_x = train_set[0]
train_y = to_categorical(train_set[1], 10)
test_x = test_set[0]
test_y = to_categorical(test_set[1],10)

# let's see result of categorizing the outputs
print(test_y[:5])

[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [23]:
from keras.models import Sequential

model = Sequential()

In [24]:
from keras.layers import Dense, Activation

model.add(Dense(units=250, input_dim=784))
model.add(Activation('relu'))
model.add(Dense(units=100))
model.add(Activation('relu'))
model.add(Dense(units=10))
model.add(Activation('softmax'))

In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [26]:
model.fit(train_x, train_y, epochs=5, batch_size=32)

Epoch 1/5
50000/50000 [==============================] - 6s - loss: 0.6491 - acc: 0.8295       

In [27]:
loss_and_metrics = model.evaluate(test_x, test_y, batch_size=128)

 9344/10000 [===========================>..] - ETA: 0s